# 03 正则表达式与HTML解析库的使用

内容导航：

1. 正则表达式
2. BeautifulSoup
3. XPath

## 3.1 正则表达式

在编写处理网页文本的程序时，经常会有查找符合某些复杂规则的字符串的需要。**正则表达式(Regular Expression)** 就是用于描述这些规则的工具。
正则表达式是由普通字符（例如字符a到z）以及特殊字符（称为"元字符"）组成的文字模式(patterns)。模式用于描述在搜索文本时要匹配的一个或多个字符串。正则表达式作为一个模板，将某个字符模式与所搜索的字符串进行匹配。

### 基本语法与使用

最简单的正则表达式：由普通的字符构成的字符串

In [2]:
import re

pattern = re.compile(r'we', re.I)
text = 'WewEwewellwelcome'
result = re.match(pattern, text)
print(result)

<re.Match object; span=(0, 2), match='We'>


In [20]:
result = re.search(pattern, text)
result.group()

'We'

In [22]:
result = re.findall(pattern, text)
result

['We', 'wE', 'we', 'we', 'we']

In [188]:
result = re.finditer(pattern, text)
[item.group() for item in result]

['We', 'wE', 'we', 'we', 'we']

In [32]:
result = re.sub(pattern, 'MN', text)
result

'MNMNMNMNllMNlcome'

In [33]:
result = re.subn(pattern, 'MN', text, 2)
result

('MNMNwewellwelcome', 2)

#### 1. 元字符

元字符主要有四个用途：

1. 匹配字符
2. 匹配位置
3. 匹配数量
4. 匹配模式

常用元字符:

|元字符|含义|
|----|:---|
| . |匹配除换行符以外的任意字符
| \b|匹配单词的开始或结束
| \d|匹配数字
| \w|匹配字母、数字、下划线或汉字
| \s|匹配任意空白字符，包括空格、制表符(tab)、换行符、中文全角空格等
| ^ |匹配字符串的开始
| $ |匹配字符串的结束

In [34]:
# 匹配所有以s开头的单词
pattern = re.compile(r'\bs\w*\b')
text = 'we are still studying and so busy.'
re.findall(pattern, text)

['still', 'studying', 'so']

#### 2. 字符转义

如果要查找元字符本身，比如查找"."或者"*"就会出问题，因为它们具有特定的功能。此时就需要转义，使用"\"来取消这些字符的特殊含义。因此，如果要查找"."、"\"或者"*"时，必须写成"\."、"\\"和"\*"。

In [38]:
# 匹配类似www.163.com这样的网址
pattern = re.compile(r'\w*\.\w*\.\w*')
text = 'www.baidu.com, www.126.com, http://www.python.org'
re.findall(pattern, text)

['www.baidu.com', 'www.126.com', 'www.python.org']

#### 3. 重复

匹配重复的限定符如下表所示：

元字符|含义
-:|-:
 * |重复零次或多次
 + |重复一次或更多次
 ？|重复零次或一次
{n}|重复n次
{n,}|重复至少n次
{n,m}|重复n到m次

In [41]:
# 匹配6～12个数字的字符串
pattern = re.compile(r'\d{6,12}')
text = '1234, 123456, ab123456, 12345678ab, 65432102'
re.findall(pattern, text)

['123456', '123456', '12345678', '65432102']

#### 4. 字符集合

正则表达式通过`[]`实现自定义字符集合

In [48]:
# 匹配由小写字符开始，后接6位数字，以“."结束的字符串
pattern = re.compile(r'[a-z]\d{6}\.')
text = 'A123 a123456 b234567. c3456 f123456.'
re.findall(pattern, text)

['b234567.', 'f123456.']

#### 5. 分支条件

正则表达式的分支条件是指有几种匹配规则，如果满足其中任何一种规则都应该当成匹配。具体语法是用"|"把不同的规则分隔开。

In [49]:
# 匹配电话号码（3位区号-8为本地号，或者4位区号-7位本地号）
pattern = re.compile(r'0\d{2}-\d{8}|0\d{3}-\d{7}')
text = '7768777, 0352-8324115, 010-55787876, 12-34567890'
re.findall(pattern, text)

['0352-8324115', '010-55787876']

#### 6. 分组

正则表达式使用括号"()"实现分组，将其当作一个整体。

In [3]:
# 匹配类似www.163.com这样的网址
pattern = re.compile(r'((\w*\.){2}\w*)')
text = 'www.163.com, 111.222.444'
result = re.search(pattern, text)

In [4]:
result.group(0)

'www.163.com'

#### 7. 反义

有时需要查找除某一类字符集之外的字符，就需要用到反义，如下表所示：

元字符|含义
-:|-:
 \W |匹配任意不是字母、数字、下划线及汉字的字符
 \S |匹配任意不是空白字符的字符
 \D |匹配任意非数字的字符
 \B |匹配不是单词开头或结束的位置
 `[^a]`|匹配除了a以外的任意字符
 `[^abcde]`|匹配除了a,b,c,d,e以外的任意字符

#### 8. 贪婪模式与懒惰模式

总的来说，贪婪模式的核心点就是尽可能多地匹配，而懒惰模式的核心点就是尽可能少地匹配。

In [60]:
pattern = re.compile(r'p.*y') # 贪婪模式
text = 'abcdfphp345pythony_py'
result = re.search(pattern, text)
result

<re.Match object; span=(5, 21), match='php345pythony_py'>

In [61]:
pattern = re.compile(r'p.*?y') # 懒惰模式
result = re.search(pattern, text)
result

<re.Match object; span=(5, 13), match='php345py'>

## 3.2 BeautifulSoup

Beautiful Soup是一个可以从HTML或XML文件中提取数据的Python库。它能够实现文档导航和查找等功能。

In [92]:
from bs4 import BeautifulSoup
import requests

url = "https://www.runoob.com/"
user_agent = 'Mozilla/5.0 (Macintosh;\
Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/75.0.3770.100 Safari/537.36'
headers = {'User-Agent': user_agent}
r = requests.get(url, headers = headers)
r.encoding = 'utf-8'
soup = BeautifulSoup(r.text, 'lxml')

In [94]:
# print(soup.prettify())

### 对象种类

BeautifulSoup将复杂HTML文档转换为一个复杂的树形结构，每个节点都是Python对象，可归为以下4种：

#### Tag对象

In [95]:
print(soup.name)

[document]


In [96]:
print(soup.title.name)

title


In [97]:
print(soup.title.text)

菜鸟教程 - 学的不仅是技术，更是梦想！


In [98]:
print(soup.p)

<p>感谢您的支持，我会继续努力的!</p>


In [99]:
soup.p

<p>感谢您的支持，我会继续努力的!</p>

In [134]:
# soup.head.contents

In [105]:
soup.head.children

In [135]:
#for child in soup.head.children:
#    if child!='\n':
#        print(child)

#### NavigableString

In [127]:
type(soup.p.string)

bs4.element.NavigableString

In [128]:
soup.p.string

'感谢您的支持，我会继续努力的!'

In [111]:
soup.html.string

In [112]:
soup.head.strings

<generator object Tag._all_strings at 0x1105058b8>

In [136]:
#for s in soup.strings:
#    if s!='\n':
#        print(s.strip())

In [137]:
#for s in soup.stripped_strings:
#    print(s)

#### BeautifulSoup

BeautifulSoup对象表示的是文档的全部内容。

In [129]:
type(soup)

bs4.BeautifulSoup

In [130]:
soup.name

'[document]'

In [119]:
print(soup.title)

<title>菜鸟教程 - 学的不仅是技术，更是梦想！</title>


### 遍历文档树

#### 子节点

In [140]:
# soup.head.contents

#### 父节点

In [122]:
print(soup.title.parent.name)

head


In [124]:
print(soup.a)

<a href="/">菜鸟教程 -- 学的不仅是技术，更是梦想！</a>


In [125]:
for parent in soup.a.parents:
    if parent is None:
        print(parent)
    else:
        print(parent.name)

h1
div
div
div
body
html
[document]


### 搜索文档树

In [141]:
help(soup.find_all)

Help on method find_all in module bs4.element:

find_all(name=None, attrs={}, recursive=True, text=None, limit=None, **kwargs) method of bs4.BeautifulSoup instance
    Extracts a list of Tag objects that match the given
    criteria.  You can specify the name of the Tag and any
    attributes you want the Tag to have.
    
    The value of a key-value pair in the 'attrs' map can be a
    string, a list of strings, a regular expression object, or a
    callable that takes a string and returns whether or not the
    string matches for some custom definition of 'matches'. The
    same is true of the tag name.



### CSS选择器

In [142]:
help(soup.select)

Help on method select in module bs4.element:

select(selector, namespaces=None, limit=None, **kwargs) method of bs4.BeautifulSoup instance
    Perform a CSS selection operation on the current element.
    
    This uses the SoupSieve library.
    
    :param selector: A string containing a CSS selector.
    
    :param namespaces: A dictionary mapping namespace prefixes
    used in the CSS selector to namespace URIs. By default,
    Beautiful Soup will use the prefixes it encountered while
    parsing the document.
    
    :param limit: After finding this number of results, stop looking.
    
    :param kwargs: Any extra arguments you'd like to pass in to
    soupsieve.select().



In [149]:
links = soup.select('a.item-top.item-1')
[link.h4.text.strip() for link in links]

['【学习 HTML】',
 '【学习 HTML5】',
 '【学习 CSS】',
 '【学习 CSS3】',
 '【学习 Bootstrap3】',
 '【学习 Bootstrap4】',
 '【学习 Font Awesome】',
 '【学习 Foundation】',
 '【学习 JavaScript】',
 '【学习 HTML DOM】',
 '【学习 jQuery】',
 '【学习 AngularJS】',
 '【学习 AngularJS2】',
 '【学习 Vue.js】',
 '【学习 React】',
 '【学习 TypeScript】',
 '【学习 jQuery UI】',
 '【学习 jQuery EasyUI 】',
 '【学习 Node.js】',
 '【学习 AJAX】',
 '【学习 JSON】',
 '【学习 Highcharts】',
 '【学习 Google 地图】',
 '【学习 PHP】',
 '【学习 Python】',
 '【学习 Python3】',
 '【学习 Django】',
 '【学习 Linux】',
 '【学习 Docker】',
 '【学习 Ruby】',
 '【学习 Java】',
 '【学习 C】',
 '【学习 C++】',
 '【学习 Perl】',
 '【学习 Servlet 】',
 '【学习 JSP】',
 '【学习 Lua】',
 '【学习 Scala】',
 '【学习 Go】',
 '【设计模式】',
 '【正则表达式】',
 '【学习 Maven】',
 '【学习 NumPy】',
 '【学习 ASP】',
 '【学习 AppML】',
 '【学习 VBScript】',
 '【学习 SQL】',
 '【学习 Mysql】',
 '【学习 PostgreSQL】',
 '【学习 SQLite】',
 '【学习 MongoDB】',
 '【学习 Redis】',
 '【学习 Memcached】',
 '【学习 Android】',
 '【学习 Swift】',
 '【学习 jQuery Mobile】',
 '【学习 ionic】',
 '【学习 Kotlin】',
 '【学习 XML】',
 '【学习 DTD】',
 '【学习 XML DOM】',
 '【学习 XSLT】',
 '【学

## 3.3 lxml的XPath解析

BeautifulSoup可以将lxml作为默认的解析器使用，同样lxml可以单独使用。
区别：
1. BeautifulSoup和lxml的原理不同，前者基于DOM的，会载入整个文档，解析整个DOM树，因此时间和内存开销较大。lxml使用XPath技术查询和处理HTML/XML，只进行局部遍历，速度较快，不过当前者使用lxml作为默认解析库时，二者性能差别不大。
2. BeautifulSoup使用简单，API人性化，支持CSS选择器，适合新手。lxml的XPath写起来比较麻烦，开发效率不如前者。

In [150]:
from lxml import etree

In [151]:
html = etree.HTML(r.text)

In [152]:
html

<Element html at 0x11052cb08>

In [163]:
result = etree.tostring(html, encoding="utf-8", pretty_print=True)
#help(etree.tostring)

In [166]:
# print(result.decode('utf-8'))

In [181]:
links = html.xpath('.//a[@class="item-top item-1"]')
#help(links[0])
for link in links:
    print(link.get('href'))

//www.runoob.com/html/html-tutorial.html
//www.runoob.com/html/html5-intro.html
//www.runoob.com/css/css-tutorial.html
//www.runoob.com/css3/css3-tutorial.html
//www.runoob.com/bootstrap/bootstrap-tutorial.html
//www.runoob.com/bootstrap4/bootstrap4-tutorial.html
//www.runoob.com/font-awesome/fontawesome-tutorial.html
//www.runoob.com/foundation/foundation-tutorial.html
//www.runoob.com/js/js-tutorial.html
//www.runoob.com/htmldom/htmldom-tutorial.html
//www.runoob.com/jquery/jquery-tutorial.html
//www.runoob.com/angularjs/angularjs-tutorial.html
//www.runoob.com/angularjs2/angularjs2-tutorial.html
//www.runoob.com/vue2/vue-tutorial.html
//www.runoob.com/react/react-tutorial.html
//www.runoob.com/typescript/ts-tutorial.html
//www.runoob.com/jqueryui/jqueryui-tutorial.html
//www.runoob.com/jeasyui/jqueryeasyui-tutorial.html
//www.runoob.com/nodejs/nodejs-tutorial.html
//www.runoob.com/ajax/ajax-tutorial.html
//www.runoob.com/json/json-tutorial.html
//www.runoob.com/highcharts/highcharts